In [24]:
#!/usr/bin/env python
# coding: utf-8

# In[74]:


# -*- coding: utf-8 -*-
import os
import cgi
import re
import ast
import time
import random
import urllib.request
import requests
from lib import init_log, handle_headers, FatalBox, goto

# In[81]:


# 设置日志文件为同目录的dl.log
log = init_log(goto("dl.log"))
here = goto("")
# 设置工作目录
os.chdir(here)

# 发送请求时，headers假装自己是浏览器
headers = """
User-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"""


class ContractDownloader():

    def __init__(self, cookie, headers, interval_each_c=(2, 3), interval_each_dl=(0.5, 1)):
        # 浏览器cookie, header， 每个合同处理间隔时间， 同一个合同的每个文件下载间隔, 合同储存方法
        self.headers = handle_headers(headers)
        self.headers.update({"cookie": cookie})
        self.interval_each_c = interval_each_c
        self.interval_each_dl = interval_each_dl
        self.downloaded = []
        self.downloader = self._downloader()

    def search_contract(self, contract_no):
        """ 根据合同号查询 """
        if not contract_no:
            raise FatalBox

        # 发起请求，根据合同号查询合同的contract_id
        url = "https://ls.cf-finance.com/hlsprod/autocrud/cont.CON555.con_contract_for_query/query?pagesize=13&pagenum=1&_fetchall=false&_autocount=true"
        headers = self.headers
        r = requests.post(url, headers=headers,
                          data={'_request_data': '{"parameter":{"contract_number":"%s","authority_flag":"Y","trx_category":"CONTRACT","trx_id":"t1.contract_id"}}' % contract_no})
       
        res = r.json()

        # 判断查询结果是否正常
        if res["success"] is not True:
            log.warning("查询订单号出错： %s" % contract_no)
            log.warning("查询订单号出错： %s" % res)
            raise FatalBox
        if not isinstance(res["result"]["record"], dict):
            log.warning("查询订单号出错： %s" % contract_no)
            raise FatalBox

        c = res["result"]["record"]
        if c.get("contract_number") == contract_no:
            if not c.get("contract_id"):
                log.warning("合同缺少contract_id")
                return
            # 查询成功， 返回该合同数据， 是个dict
            return c

        # 程序如果走到这里说明没查到
        log.warning("没查询到相关合同： %s, res: %s" % (contract_no, res))

        
        
    def download_contract_attachments(self, contract_no, contract):
        """ 根据contract_id下载对应文件 """

        # 从合同数据中取出contract_id
        contract_id = contract.get("contract_id")

        log.info("下载：%s" % contract_no)
        dl_url = "https://ls.cf-finance.com/hlsprod/modules/cont/CON555/lease_atm_batch_dl.svc?table_name=CON_CONTRACT&table_pk_value={}&doc_code={}".format(
            contract_id,  contract_no)
        self.save_contract_attachment(contract_no, dl_url, filename=contract_no)
        self.downloaded.append(contract_no)

            # 随机等待xx秒下载同一个合同的不同文件， 通过interval_each_dl，调整等待时间
        time.sleep(random.uniform(*self.interval_each_dl))

    def get_contract_nos(self):
        # 打开contracts.txt文件，读取所有合同号
        with open("./contracts.txt", "r") as f:
            return map(lambda x: x.strip(), filter(None, f.read().split("\n")))


    def stat(self, line, tag=False):
        """往stat里面写数据"""

        with open("./stat.txt", "a+",encoding='utf-8') as f:
            if tag:
                f.write("\n" * 3 + "又一次运行的起点".center(100, "-"))
            f.write(line)

    def save_contract_attachment(self, contract_no, url, filename, contracts_dir=here + "/contracts02"):
        """ 保存文件到电脑 """

        if not contract_no:
            log.warning("合同号为空，无法保存")
            raise FatalBox
        
        dirpath = contracts_dir.rstrip("/")
        if not os.path.isdir(dirpath):
            os.mkdir(dirpath)
        if url:
            remotefile = urllib.request.urlopen(url)
            blah = remotefile.info()
            value, params = cgi.parse_header(blah['Content-Disposition'])
            name = params["filename"]
            nofile = False
            try:
                # 观察发现空压缩包都是22
                nofile = int(blah.get("Content-Length")) == 22
            except Exception:
                pass
            if name and not nofile:
                log.info("下载文件： %s" % name)
                self.downloader.urlretrieve(url, "%s/%s" % (dirpath, name))
            else:
                log.info("无相关文件")

    def _downloader(self):
        opener = urllib.request.build_opener()
        opener.addheaders = list(self.headers.items())
        urllib.request.install_opener(opener)
        return urllib.request

    def run(self):
        # 入口方法

        # get_contract_nos方法从contracts.txt中得到所有的合同号
        # for循环处理每一个合同
        for i, contract_no in enumerate(self.get_contract_nos()):
            log.info("处理合同： %s" % contract_no)
            # 初始化
            self.downloaded = []
            contract = None
            try:
                # 尝试查询合同号
                contract = self.search_contract(contract_no)
                if contract is None:
                    raise FatalBox
            except FatalBox:
                log.warning("查询出错：%s" % contract_no)
            except Exception as ex:
                log.warning("新的致命错误，请处理：%s" % contract_no)
                raise ex

            # 没查到该合同号
            if contract is None:
                log.warning("查询出错2：%s" % contract_no)
            else:
                try:
                    self.download_contract_attachments(contract_no, contract)
                except FatalBox:
                    log.exception("下载出错：%s" % contract_no)
                except Exception as ex:
                    log.warning("新的致命错误2，请处理：%s" % contract_no)
                    raise ex
            log.info("处理结束\n")

            # 记录该合同文件下载情况
            self.stat("\n%s: %s, %s" % (contract_no, len(self.downloaded), ", ".join(sorted(self.downloaded))), i==0)

            # 随机等待xx秒下载下一个合同的相关文件， 通过interval_each_c，调整等待时间
            time.sleep(random.uniform(*self.interval_each_c))



In [26]:
# # In[76]:


if __name__ == "__main__":
    with open(here + "/cookies", "rb") as f:
        cookies = f.read()
        ContractDownloader(cookies,
                           headers=headers,
                           interval_each_c=(0.3, 0.5),  # 随机等待0.3~0.5秒下载下一个合同的相关文件
                           interval_each_dl=(0.1, 0.3),  # 随机等待0.2~0.4秒下载同一个合同的不同文件
                           ).run()


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-89: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-89: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-89: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-89: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-87: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

...

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-89: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-89: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-88: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-88: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Xin Chen\Anaconda3\lib\logging\__init__.py", line 994, in emit
    stream.write(msg)
  File "C:\Users\Xin Chen\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 85-88: character maps to <undefined>
Call stack:
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Xin Chen\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Xin Chen\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in star